# GANs with non image data 


- [Robin Ricard's GAN for sinusoid modeling (keras)](http://www.rricard.me/machine/learning/generative/adversarial/networks/keras/tensorflow/2017/04/05/gans-part2.html)
- [A methodology to transform non-image data to an image for convolution NN architecture](https://www.nature.com/articles/s41598-019-47765-6)
- [reddit: GAN w/ non image data](https://www.reddit.com/r/MachineLearning/comments/7ac6xx/d_gan_with_non_image_data/)
- Real-values (medical) time series generation w/ recurrent conditional GANs - [[paper]](https://arxiv.org/pdf/1706.02633.pdf), [[code]](https://github.com/ratschlab/RGAN)

In [1]:
%matplotlib inline
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
import tensorflow as tf
from tensorflow import keras

# from keras.models import Model
# from keras.layers import Input, Reshape

# "core" layers in Keras are those used in almost every NN
from keras.layers.core import Dense, Activation, Dropout, Flatten
# synonymous to tf.keras.layers:
# from keras.layers import Dense, Activation, Dropout, Flatten


# from keras.layers.normalization import BatchNormalization
# from keras.layers.convolutional import UpSampling1D, Conv1D
# from keras.layers.advanced_activations import LeakyReLU
# import keras.optimizers as optim
# from keras.callbacks import TensorBoard

In [2]:
def get_generative(G_in, dense_dim=200, out_dim=50, lr=1e-3):
    x = keras.layers.Dense(dense_dim)(G_in)
    x = keras.layers.Activation('tanh')(x)
    G_out = Dense(out_dim, activation='tanh')(x)
    G = keras.models.Model(G_in, G_out)
    optimizer = keras.optimizers.SGD(lr=lr)
    G.compile(loss='binary_crossentropy', optimizer=optimizer)
    return G, G_out

G_in = keras.layers.Input(shape=[10])
G, G_out = get_generative(G_in)
G.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
dense (Dense)                (None, 200)               2200      
_________________________________________________________________
activation (Activation)      (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                10050     
Total params: 12,250
Trainable params: 12,250
Non-trainable params: 0
_________________________________________________________________


In [3]:
G_out.dtype

tf.float32

In [4]:
G_out.shape

TensorShape([None, 50])

In [5]:
G_out.op

<tf.Operation 'dense_1/Tanh' type=Tanh>

### TensorFlow Notes

#### `keras.layers.Dense` (class): 
- A  densely-connected NN layer. Synounymous to fully-connected layer in torch. Performs a linear operation on the layer’s input vector, `output = activation(dot(input, kernel) + bias)`

#### `keras.models.Model` (class):
- Groups layers into an object with training and inference features. 
- Args:
  - `inputs`: The input(s) of the model: a `keras.Input` object or list of `keras.Input` objects.
  - `outputs`: The output(s) of the model. See Functional API example below.
  - `name`: String, the name of the model.
- If `model = keras.models.Model(...)`, you can use `model.compile()` to config the model with losses and metrics,
`model.fit()` to train, and `model.predict()` to make predictions.

#### `keras.layers.Activation`
- Applies an activation function to an output.
- Arguments:
  - activation: Activation function, such as `tf.nn.relu`, or string name of built-in activation function, such as "relu".
- Usage:
```python
>>> layer = tf.keras.layers.Activation('relu')
>>> output = layer([-3.0, -1.0, 0.0, 2.0])
>>> list(output.numpy())
[0.0, 0.0, 0.0, 2.0]
>>> layer = tf.keras.layers.Activation(tf.nn.relu)
>>> output = layer([-3.0, -1.0, 0.0, 2.0])
>>> list(output.numpy())
[0.0, 0.0, 0.0, 2.0]
```